In [1]:
import sys
print(sys.executable)

/opt/homebrew/Cellar/jupyterlab/4.4.0_1/libexec/bin/python


In [2]:
!{sys.executable} -m pip install opencv-python ultralytics

  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached ultralytics-8.3.230-py3-none-any.whl.metadata (37 kB)
  Using cached polars-1.35.2-py3-none-any.whl.metadata (10 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached polars_runtime_32-1.35.2-cp39-abi3-macosx_11_0_arm64.whl.metadata (1.5 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 9.0 MB/s eta 0:00:008.8 MB/s eta 0:00:01
Using cached ultralytics-8.3.230-py3-none-any.whl (1.1 MB)
Using cached polars-1.35.2-py3-none-any.whl (783 kB)
Using cached polars_runtime_32-1.35.2-cp39-abi3-macosx_11_0_arm64.whl (36.7 MB)
Using cached ultralytics_thop-2.0.18-py3-none-any.whl (28 kB)
  Attempting uninstall: numpy━━━━━━━━━━━━━━━━━━━━━━━━━ 0/6 [polars-runtime-32]
    Found existing installation: numpy 2.3.2━━━━━━ 0/6 [polars-runtime-32]
    Uninstalling numpy-2.3.2:━━━━━

In [3]:
!{sys.executable} -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.9 MB/s eta 0:00:006 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


In [3]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

# Input video
input_path = "sample_queue2.mp4"   # <- your downloaded video
cap = cv2.VideoCapture(input_path)

if not cap.isOpened():
    print("Error: could not open video")
    cap.release()
    raise SystemExit

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Fallback if fps is 0 for some reason
if fps == 0:
    fps = 30.0

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # good for .mp4 on macOS
output_path = "queue_annotated2.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        # End of video
        break

    frame_count += 1

    # Run detection
    results = model(frame, verbose=False)
    people_in_line = 0

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            if cls_id != 0:  # only persons
                continue

            x1, y1, x2, y2 = box.xyxy[0]
            people_in_line += 1

            # draw bbox
            cv2.rectangle(
                frame,
                (int(x1), int(y1)),
                (int(x2), int(y2)),
                (0, 255, 0),
                2
            )

    # Draw count in bottom-left
    text = f"People in line: {people_in_line}"
    cv2.putText(
        frame,
        text,
        (10, h - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.0,
        (0, 255, 0),
        2,
        cv2.LINE_AA,
    )

    # Write annotated frame to output video
    out.write(frame)

cap.release()
out.release()

print(f"Done! Saved annotated video to: {output_path}")


Done! Saved annotated video to: queue_annotated2.mp4
